In [ ]:
import pandas as pd
import geopandas as gpd

pd.options.plotting.backend = "plotly"
import numpy as np
import plotly.graph_objects as go


from ares_py.class_project import Project
from ares_py.tools.get_ld import get_ld

In [ ]:
project_name = "kysuce"

grid_skip = [1, 2, 3, 4, 5, 6]
crs = 8353
res_range = [1, 1000]


atlas_input = dict(
    res_min=[0] * 6,
    res_max=[500] * 6,
    page_heights=[420] * 6,
    map_scales=[500] * 6,
    lvl_frequency=6 * [3],
    z_gap=[20, 15, 15, 25, 25, 25],
)
prj = Project(project_name, crs=crs, res_range=res_range)

In [ ]:
project_name = "dd_lavka"

grid_skip = [1, 2, 3, 4, 5]
crs = 8353
res_range = [100, 10000]


atlas_input = dict(
    res_min=[100] * 5,
    res_max=[20000] * 5,
    page_heights=[594, 420, 420, 420, 420],
    # page_heights_int=[420] + [297] * 4,
    page_heights_int=[420] * 5,
    map_scales=[1000] * 5,
    lvl_frequency=5 * [1],
    z_gap=[30, 50] + [30] * 3,
)
prj = Project(project_name, crs=crs, res_range=res_range)

In [ ]:
project_name = "budmerice"

grid_skip = []
crs = 8353
res_range = [1, 10000]


atlas_input = dict(
    res_min=[0] * 4,
    res_max=[500] * 4,
    page_heights=[420, 594, 420, 594],
    page_heights_int=[297] * 4,
    map_scales=[250] * 4,
    lvl_frequency=[1, 3, 1, 3],
    z_gap=[10, 10, 10, 5],
)
prj = Project(project_name, crs=crs, res_range=res_range)

In [ ]:
prj = Project(project_name, crs=crs, res_range=res_range)

In [ ]:
from ares_py.class_raw import Raw


prj = prj.Process_raw()

print("\n")
prj = prj.Process_apy()
prj = prj.Merge_apy_coordinates()
print("\n")

In [ ]:
prj = prj.Gpkg_coords()


prj = prj.Gpkg_measured()

prj = prj.Gpkg_mask(manual_overwrite=False)
prj = prj.Export_r2d()

prj = prj.Gpkg_atlas(atlas_input)

In [ ]:
prj = prj.Export_gridding_input()
# prj = prj.Grid_data(skip=grid_skip, cell_size=0.5)

In [ ]:
prj = prj.Gpkg_contours(interval=10)

In [ ]:
prj.QC_lines()